In [ ]:
!pip install --upgrade huggingface_hub==1.0.1
import pandas as pd
df = pd.read_json("hf://datasets/llm-book/japanese-wikipedia/ja_wiki.jsonl", nrows=1000, lines=True)

In [ ]:
df.head(5)

In [ ]:
offset = 10
for i in range(100):
    query = "京都"
    text = df.iloc[i]["text"]
    if query in text:
        print(f"================記事インデックス{i}================")
        char_index = text.index(query)
        print(text[char_index-offset:char_index+offset])


In [ ]:
!pip install mecab-python3==1.0.10 unidic-lite==1.0.8

# 形態素解析の設定
import MeCab
tagger = MeCab.Tagger("-Owakati")

# 試してみる
result=tagger.parse("私は京都に住んでいます")
print(result)

In [ ]:
offset = 10
for i in range(100):
    query = "京都"
    text = df.iloc[i]["text"]
    text = tagger.parse(text) #形態素解析した結果で置き換える
    if query in text:
        print(f"================記事インデックス{i}================")
        char_index = text.index(query)
        print(text[char_index-offset:char_index+offset])

In [ ]:
!pip install rank_bm25==0.2.2

In [ ]:
from rank_bm25 import BM25Okapi
import MeCab
import numpy as np

# MeCabで分かち書き
tagger = MeCab.Tagger("-Owakati")
tokenized_corpus = [tagger.parse(doc).strip().split() for doc in df["text"]]

# BM25 モデル作成
bm25 = BM25Okapi(tokenized_corpus)

# クエリを分かち書きしてスコア計算
query = "東京　音楽"
tokenized_query = tagger.parse(query).strip().split()
scores = bm25.get_scores(tokenized_query)


# 上位5件のインデックスを取得
top_n = 5
top_indices = np.argsort(scores)[::-1][:top_n]

# 結果表示
print("Top 5 documents:")
for idx in top_indices:
    print(f"======== Doc {idx}: Score = {scores[idx]:.4f} =======")
    print(f"Text = {df.iloc[idx].text[:100]}")